

# Solving Randomized PDEs on Random Geometries

This notebook demonstrates the generation, visualization, and solution of partial differential equations (PDEs) on randomly generated geometries using isogeometric analysis (IGA).

The main steps include:

- Creating a random 2D geometry using B-spline surfaces.
- Applying random boundary conditions to the geometry.
- Setting up and solving a chosen PDE (such as Poisson or Helmholtz) on the generated domain.
- Visualizing the geometry, boundary conditions, and resulting solution field.

The workflow is meant for experiments with geometry uncertainty, testing solvers, and visualizing solution behaviors on complex domains. All steps are customizable and reproducible via fixed random seeds.



In [ ]:
# Single Sample PDE Generation and Visualization
# This notebook generates a single sample of a random geometry and solves a PDE on it

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from scipy.fft import fft2, ifft2
from pyiga import bspline, assemble, geometry, vis
import scipy.sparse.linalg
import os
import torch
from geniga.geometry.random import generate_random_geom, generate_random_bcs
from geniga.physics.validation import (
    compute_poisson_residual,
    compute_helmholtz_residual
)

# Set random seed for reproducibility (optional)
#np.random.seed(42)

print("Imports successful!")


### Parameters

**Key categories of parameters:**

- **Geometry and Discretization:**
  - `N`: Grid resolution as a tuple `(nx, ny)`, specifying the number of grid points in each spatial direction.
  - `deg`: The degree of the B-spline basis functions used for isogeometric analysis.
- **PDE Type:**
  - `equation`: Selects which PDE to solve; options include `"poisson"` or `"helmholtz"`.
- **Random Geometry Control:**
  - `epsilon_strain`: Controls the magnitude of random strains (deformations) applied to the geometry.
  - `epsilon_rotation`: Controls the magnitude of random rotational perturbations.
- **Boundary Condition Randomization:**
  - `bc_scale`: The amplitude (scaling factor) of the random boundary condition function.
  - `bc_sigma`: The spatial scale (standard deviation) for smoothing random fluctuations on the boundary.
- **Helmholtz Equation Parameter:**
  - `k`: The wave number for the Helmholtz equation (relevant only if `equation="helmholtz"`).

Adjusting these parameters allows you to experiment with different random domains, boundary effects, and PDE types.


In [ ]:
# Parameters for geometry and PDE
N = (64, 64)  # Grid resolution
deg = 2       # B-spline degree
equation = "helmholtz"  # Type of PDE to solve ("poisson" or "helmholtz")

# Parameters for random geometry generation
epsilon_strain = 4.0
epsilon_rotation = 4.0

# Parameters for boundary conditions
bc_scale = 2.0
bc_sigma = 16.0/2.0

# Parameters for Helmholtz equation (only used if equation="helmholtz")
k = 16.0  # Wave number

print(f"Grid resolution: {N}")
print(f"B-spline degree: {deg}")
print(f"Equation type: {equation}")
if equation == "helmholtz":
    print(f"Wave number k: {k}")
print(f"Strain epsilon: {epsilon_strain}")
print(f"Rotation epsilon: {epsilon_rotation}")


### Generate the geometry

A random IGA parametrization is generated in the next cell.

In [ ]:
# Generate random geometry
print("Generating random geometry...")
geo, spline_space, X_new, Y_new = generate_random_geom(
    N=N, 
    deg=deg, 
    epsilon_strain=epsilon_strain, 
    epsilon_rotation=epsilon_rotation
)

print(f"Generated geometry with {geo.coeffs.shape} control points")
print(f"Spline space dimensions: {[kvs.numdofs for kvs in spline_space]}")

# Display some information about the geometry
print(f"X coordinates range: [{X_new.min():.3f}, {X_new.max():.3f}]")
print(f"Y coordinates range: [{Y_new.min():.3f}, {Y_new.max():.3f}]")


### Boundary conditions generation

We generate random Dirichlet boundary conditions across the entire boundary


In [ ]:
# Generate boundary conditions
print("Generating boundary conditions...")
bcs = generate_random_bcs(geo, spline_space, scale=bc_scale, sigma=bc_sigma)

print(f"Number of Dirichlet boundary points: {len(bcs[0])}")
print(f"Boundary values range: [{bcs[1].min():.3f}, {bcs[1].max():.3f}]")

# The bcs object contains:
# bcs[0]: indices of boundary degrees of freedom
# bcs[1]: values at those boundary points


### Sovle the PDE

Assemble the system matrix, apply the boundary conditions to the linear system and then solve the system and inflate it back.
Both Helmholtz and Poisson variants are implemented.

In [ ]:
# Solve the PDE
print("Setting up and solving the PDE...")

# Define the right-hand side (source term) - in this case, zero
rhs = assemble.inner_products(spline_space, lambda x, y: 0 * x, f_physical=True, geo=geo).ravel()

if equation == "poisson":
    # Assemble stiffness matrix for Poisson equation: -∇²u = f
    A = assemble.stiffness(spline_space, geo)
    print(f"Stiffness matrix shape: {A.shape}")
    print(f"Matrix sparsity: {A.nnz / (A.shape[0] * A.shape[1]):.4f}")
    
    # Create restricted linear system with boundary conditions
    LS = assemble.RestrictedLinearSystem(A, rhs, bcs)
    print(f"Restricted system size: {LS.A.shape}")
    
    # Solve the linear system
    u_restricted = scipy.sparse.linalg.spsolve(LS.A, LS.b)
    u_full = LS.complete(u_restricted)
    
    print(f"Solution computed successfully!")
    print(f"Solution range: [{u_full.min():.6f}, {u_full.max():.6f}]")

elif equation == "helmholtz":
    # Assemble matrices for Helmholtz equation: -∇²u + k²u = f
    A = assemble.stiffness(spline_space, geo)  # Stiffness matrix (-∇² term)
    M = assemble.mass(spline_space, geo)       # Mass matrix (k²u term)
    print(f"Stiffness matrix shape: {A.shape}")
    print(f"Mass matrix shape: {M.shape}")
    print(f"Stiffness matrix sparsity: {A.nnz / (A.shape[0] * A.shape[1]):.4f}")
    print(f"Mass matrix sparsity: {M.nnz / (M.shape[0] * M.shape[1]):.4f}")
    print(f"Wave number k: {k}")
    
    # Combine matrices: -A + k²M (note: stiffness matrix already has negative sign)
    combined_matrix = -A + k**2 * M
    
    # Create restricted linear system with boundary conditions
    LS = assemble.RestrictedLinearSystem(combined_matrix, rhs, bcs)
    print(f"Restricted system size: {LS.A.shape}")
    
    # Solve the linear system
    u_restricted = scipy.sparse.linalg.spsolve(LS.A, LS.b)
    u_full = LS.complete(u_restricted)
    
    print(f"Solution computed successfully!")
    print(f"Solution range: [{u_full.min():.6f}, {u_full.max():.6f}]")
    
else:
    raise ValueError(f"Unknown equation: {equation}")

print(u_full.shape, geo.coeffs.shape)
# Create B-spline function from solution
u_func = geometry.BSplineFunc(spline_space, u_full)


### Plot the solutions

In [ ]:
# Configure matplotlib for better plots
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 12

# Plot 1: Geometry (mesh)
plt.figure(figsize=(8, 6))
plt.title('Generated Random Geometry', fontsize=14, fontweight='bold')
vis.plot_geo(geo, grid=32)
plt.xlabel('X coordinate')
plt.ylabel('Y coordinate')
plt.grid(True, alpha=0.3)
plt.axis('equal')

# Save geometry plot
geometry_filename = "geometry.png"
plt.savefig(geometry_filename, dpi=300, bbox_inches='tight')
print(f"Geometry plot saved as: {geometry_filename}")
plt.show()

# Plot 2: Solution field
plt.figure(figsize=(8, 6))
equation_title = f"{equation.capitalize()} Equation"
if equation == "helmholtz":
    equation_title += f" (k={k})"
plt.title(f'{equation_title} Solution Field', fontsize=14, fontweight='bold')
vis.plot_field(u_func, geo)
plt.xlabel('X coordinate')
plt.ylabel('Y coordinate')
plt.colorbar()
plt.axis('equal')

# Save solution plot
solution_filename = f"solution_{equation}.png"
plt.savefig(solution_filename, dpi=300, bbox_inches='tight')
print(f"Solution plot saved as: {solution_filename}")
plt.show()



### Compute the residuals and plot them

In [ ]:
# Residual check - verify the solution accuracy
print("\n" + "="*50)
print("RESIDUAL CHECK")
print("="*50)

if equation == "poisson":
    residual = compute_poisson_residual(
        torch.tensor(u_func.coeffs),               # solution coefficients (convert to torch tensor)
        torch.tensor(geo.coeffs),                  # geometry coefficients (convert to torch tensor)
        spline_space,                # spline space tuple
        return_norm=False
    ).cpu().numpy()
elif equation == "helmholtz":
    residual = compute_helmholtz_residual(
        torch.tensor(u_func.coeffs),               # solution coefficients (convert to torch tensor)
        torch.tensor(geo.coeffs),                  # geometry coefficients (convert to torch tensor)
        k,                           # wave number (can be float or torch.Tensor)
        spline_space,                # spline space tuple
        return_norm=False
    ).cpu().numpy()
else:
    raise ValueError(f"Unknown equation: {equation}")

residual_norm = np.linalg.norm(residual)

# Check if solution is reasonable
if residual_norm < 1e-10:
    print(f"\n✓ Solution appears to be accurate (residual < 1e-10)")
elif residual_norm < 1e-6:
    print(f"\n✓ Solution appears to be reasonably accurate (residual < 1e-6)")
else:
    print(f"\n⚠ Warning: Large residual detected (residual = {residual_norm:.2e})")
    print("  This might indicate numerical issues or convergence problems.")

print("\n" + "="*50)
print("PLOTTING RESIDUAL FIELD")
print("="*50)

# Create a BSplineFunc object from the residual array
residual_func = geometry.BSplineFunc(spline_space, residual)

# Plot the residual field
plt.figure(figsize=(8, 6))
equation_title = f"{equation.capitalize()} Equation"
if equation == "helmholtz":
    equation_title += f" (k={k})"
plt.title(f'{equation_title} Residual Field', fontsize=14, fontweight='bold')
vis.plot_field(residual_func, geo)
plt.xlabel('X coordinate')
plt.ylabel('Y coordinate')
plt.colorbar(label='Residual')
plt.axis('equal')

# Save residual plot
residual_filename = f"residual_{equation}.png"
plt.savefig(residual_filename, dpi=300, bbox_inches='tight')
print(f"Residual plot saved as: {residual_filename}")
plt.show()
